# Inference


## Imports


In [ ]:
!ls ../input

csiro-biomass  csiro-models


In [ ]:
import os
import gc
import numpy as np
import pandas as pd
from PIL import Image

import cv2
import timm
import torch
import torch.nn as nn
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
print(f"PyTorch: {torch.__version__}")
print(f"Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

PyTorch: 2.8.0+cu126
Device: Tesla T4


In [ ]:
# setting device on GPU if available, else CPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_WORKERS = 0
print('Using device:', DEVICE)
print('NUM_WORKERS:', NUM_WORKERS)
print()

# Additional Info when using cuda
if DEVICE.type == 'cuda':
    # clean GPU memory
    torch.cuda.empty_cache()
    gc.collect()

    # torch.set_float32_matmul_precision('high')

    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3, 1), 'GB')

Using device: cuda
NUM_WORKERS: 0

Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [ ]:
IS_ENSEMBLE = False

IS_USE_EVA = True

HARDCODED_MODEL = ''
# 'vit_giant_patch14_dinov2.lvd142m-fold1-r2_0.8401.pt'
# 'vit_large_patch14_dinov2.lvd142m-fold4-r2_0.7603.pt'
# 'convnext_base.fb_in22k_ft_in1k-fold0-r2_0.7422.pt'

HARDCODED_MODEL = 'vit_giant_patch14_dinov2.lvd142m-fold1-r2_0.8401.pt'

BATCH_SIZE = 16

# TTA helpers
TTA_TYPES = ['id', 'hflip', 'vflip', 'rot90', 'rot180', 'rot270']
TTA_TYPES = ['id', 'hflip', 'vflip']

MODELS_PATH = '/kaggle/input/csiro-models/pytorch/default/4/'
PATH_DATA = '/kaggle/input/csiro-biomass'

PATH_TEST_CSV = os.path.join(PATH_DATA, 'test.csv')
PATH_TEST_IMG = os.path.join(PATH_DATA, 'test')

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if not filename.endswith('.jpg'):
            
            print(os.path.join(dirname, filename))
        # /kaggle/input/2head/pytorch/default/1/f1dinov2
        # /kaggle/input/csiro-models/blablabla.pt
        

/kaggle/input/csiro-biomass/sample_submission.csv
/kaggle/input/csiro-biomass/train.csv
/kaggle/input/csiro-biomass/test.csv
/kaggle/input/csiro-models/pytorch/default/3/vit_large_patch14_dinov2.lvd142m-fold4-r2_0.7603.pt
/kaggle/input/csiro-models/pytorch/default/3/swin_large_patch4_window7_224.ms_in22k_ft_in1k-fold1-r2_0.7471.pt
/kaggle/input/csiro-models/pytorch/default/3/convnext_base.fb_in22k_ft_in1k-fold0-r2_0.7422.pt


In [ ]:
# EVA02 Model Configuration
EVA02_MODEL_PATH = '/kaggle/input/eva02-biomass-regression/best_model_fold_4.pth'
EVA02_IMG_SIZE = 448
EVA02_SMOOTH_FACTOR = 0.9050
EVA02_DRY_CLOVER_MIN = 1.25
EVA02_DRY_DEAD_MIN = 1.00

In [ ]:
try:
    model = torch.load(MODELS_PATH + HARDCODED_MODEL, map_location=DEVICE, weights_only=False)
except:
    MODELS_PATH = '.' + MODELS_PATH
    PATH_DATA = '.' + PATH_DATA
    PATH_TEST_CSV = '.' + PATH_TEST_CSV
    PATH_TEST_IMG = '.' + PATH_TEST_IMG
    model = torch.load(MODELS_PATH + HARDCODED_MODEL, map_location=DEVICE, weights_only=False)

model.to(DEVICE)
model.eval()
print(f"Loaded model: {HARDCODED_MODEL}")

/usr/local/lib/python3.12/dist-packages/torch/serialization.py:1493: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn(


RecursiveScriptModule(
  original_name=TransformerInferenceWrapper
  (backbone): RecursiveScriptModule(
    original_name=VisionTransformer
    (patch_embed): RecursiveScriptModule(
      original_name=PatchEmbed
      (proj): RecursiveScriptModule(original_name=Conv2d)
      (norm): RecursiveScriptModule(original_name=Identity)
    )
    (pos_drop): RecursiveScriptModule(original_name=Dropout)
    (patch_drop): RecursiveScriptModule(original_name=Identity)
    (norm_pre): RecursiveScriptModule(original_name=Identity)
    (blocks): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=Block
        (norm1): RecursiveScriptModule(original_name=LayerNorm)
        (attn): RecursiveScriptModule(
          original_name=Attention
          (qkv): RecursiveScriptModule(original_name=Linear)
          (q_norm): RecursiveScriptModule(original_name=Identity)
          (k_norm): RecursiveScriptModule(original_name=Identity)
          (attn_

In [7]:
try:
    SIZE, MEAN, STD = model.img_size, model.mean, model.std
except:
    SIZE = 224
    MEAN = [0.485, 0.456, 0.406]
    STD = [0.229, 0.224, 0.225]

In [8]:
def get_model_params(model_name: str):
    try:
        size, mean, std = model.img_size, model.mean, model.std
    except:
        size = 224
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]

### Get best model

In [9]:
# Checkpoint discovery and loading
def parse_metric_from_filename(filename: str, keyword: str) -> float:
    """Extract val_comp_metric_img from filename like ...val_comp_metric_img=0.7129.ckpt."""
    try:
        metric_part = filename.split(keyword)[-1].split('.')[:-1]
        metric_part = '.'.join(metric_part)
        return float(metric_part.replace('.ckpt', ''))
    except Exception:
        return -float('inf')

In [10]:
best_model = ''
best_score = -float('inf')

for file in os.listdir(MODELS_PATH):
    if file.endswith('.pt'):
        metric = parse_metric_from_filename(file, 'r2_')
        if metric > best_score:
            best_score = metric
            best_model = file

# HARDCODE BEST MODEL
if HARDCODED_MODEL != '':
    best_model = HARDCODED_MODEL
print(f'Best model: {best_model} with R2: {best_score}')

Best model: vit_large_patch14_dinov2.lvd142m-fold4-r2_0.7603.pt with R2: 0.7603


## Inference on Test Set

In [11]:
# Load test CSV
test_df = pd.read_csv(PATH_TEST_CSV)
test_df = test_df[~test_df['target_name'].isin(['Dry_Total_g', 'GDM_g'])]

# Pivot to one row per image
test_pivot = test_df.pivot_table(
    index='image_path',
    aggfunc='first'
).reset_index()

print(f"Test set size: {len(test_pivot)}")
print(test_pivot.head())

Test set size: 1
              image_path                   sample_id   target_name
0  test/ID1001187975.jpg  ID1001187975__Dry_Clover_g  Dry_Clover_g


In [12]:
# Create test dataset
class BiomassTestDataset(Dataset):
    """Test dataset for inference - no targets needed."""

    def __init__(self, df: pd.DataFrame, img_dir: str, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Load image
        img_path = os.path.join(
            self.img_dir, row['image_path'].replace('test/', ''))
        image = cv2.imread(img_path)

        if image is None:
            raise FileNotFoundError(f"Cannot load image: {img_path}")

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Split into left and right patches
        h, w, c = image.shape
        mid_w = w // 2

        left_patch = image[:, :mid_w, :]
        right_patch = image[:, mid_w:, :]

        # Convert to PIL
        left_pil = Image.fromarray(left_patch)
        right_pil = Image.fromarray(right_patch)

        # Apply transforms
        if self.transform:
            left_tensor = self.transform(left_pil)
            right_tensor = self.transform(right_pil)
        else:
            left_tensor = transforms.ToTensor()(left_pil)
            right_tensor = transforms.ToTensor()(right_pil)

        return {
            'left_image': left_tensor,
            'right_image': right_tensor,
            'image_id': row['image_path'].split('/')[-1].replace('.jpg', ''),
        }

In [13]:
student_val_transform = transforms.Compose([
    transforms.Resize((SIZE, SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=MEAN, std=STD)
])

In [14]:
# Create test dataloader
test_dataset = BiomassTestDataset(
    df=test_pivot,
    img_dir=PATH_TEST_IMG,
    transform=student_val_transform
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE * 2,
    shuffle=False,
    num_workers=min(NUM_WORKERS, 4),
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"Test loader created: {len(test_loader)} batches")

Test loader created: 1 batches


### TTA Inference

In [ ]:
def apply_tta(left: torch.Tensor, right: torch.Tensor, tta: str) -> tuple[torch.Tensor, torch.Tensor]:
    """Apply flip and rotation-based TTA. Input: [B, C, H, W]"""
    if tta == 'hflip':
        return torch.flip(left, dims=[3]), torch.flip(right, dims=[3])
    if tta == 'vflip':
        return torch.flip(left, dims=[2]), torch.flip(right, dims=[2])
    if tta == 'hvflip':
        return torch.flip(left, dims=[2, 3]), torch.flip(right, dims=[2, 3])
    if tta == 'rot90':
        return torch.rot90(left, k=1, dims=[2, 3]), torch.rot90(right, k=1, dims=[2, 3])
    if tta == 'rot180':
        return torch.rot90(left, k=2, dims=[2, 3]), torch.rot90(right, k=2, dims=[2, 3])
    if tta == 'rot270':
        return torch.rot90(left, k=3, dims=[2, 3]), torch.rot90(right, k=3, dims=[2, 3])
    return left, right

In [16]:
def expand_predictions(preds_3: torch.Tensor) -> torch.Tensor:
    """
    Convert [B, 3] predictions to [B, 5] submission format.
    
    Model predicts: [Dry_Green_g, Dry_Total_g, GDM_g]
    Submission needs: [Dry_Clover_g, Dry_Dead_g, Dry_Green_g, Dry_Total_g, GDM_g]
    """
    green = preds_3[:, 0]   # Dry_Green_g (predicted)
    total = preds_3[:, 1]   # Dry_Total_g (predicted)
    gdm = preds_3[:, 2]     # GDM_g (predicted)
    
    # Calculate missing targets
    dead = total - gdm      # Dry_Dead_g = Total - GDM
    clover = gdm - green    # Dry_Clover_g = GDM - Green

    # Ensure no negative values
    dead = torch.clamp(dead, min=0.0)
    clover = torch.clamp(clover, min=0.0)
    
    
    # Return in competition order
    return torch.stack([clover, dead, green, total, gdm], dim=1)

In [ ]:
def predict_model_batch(model: torch.nn.Module, batch: dict, tta_types: list[str]) -> torch.Tensor:
    """Run model over TTA variants and average. Returns [B, 5]."""
    model_preds = []
    left = batch['left_image'].to(DEVICE)
    right = batch['right_image'].to(DEVICE)
    
    for tta in tta_types:
        left_t, right_t = apply_tta(left, right, tta)
        
        # Model returns tensor [B, 3], not dict!
        preds_3 = model(left_t, right_t)  # [B, 3] = [Green, Total, GDM]
        
        preds_3 = torch.clamp(preds_3, min=0.0)
        
        # Expand to 5 targets
        preds_5 = expand_predictions(preds_3)  # [B, 5]
        model_preds.append(preds_5)
    
    # Average across TTA variants
    return torch.stack(model_preds, dim=0).mean(dim=0)  # [B, 5]

### Ensemble Inference

In [ ]:
class ModelConfig:
    """Configuration for a single model in ensemble."""
    
    def __init__(self, model_path: str, img_size: int = None, mean: list = None, std: list = None):
        self.model_path = model_path
        self.model = None
        
        # Try to extract config from model or separate file
        self._load_model_config()
        
        # Override with provided values if any
        self.img_size = img_size or self.img_size
        self.mean = mean or self.mean
        self.std = std or self.std
    
    def _load_model_config(self):
        """Load model and extract its config."""
        try:
            self.model = torch.load(
                self.model_path, 
                map_location='cpu',
                weights_only=False
            )
            
            # Try to get config from model attributes
            try:
                self.img_size = self.model.img_size
                self.mean = self.model.mean
                self.std = self.model.std
                print(f"✅ Config loaded from model attributes")
            except AttributeError:
                # Hardcoded defaults
                self.mean = [0.485, 0.456, 0.406]
                self.std = [0.229, 0.224, 0.225]
                name = os.path.basename(self.model_path).lower()
                if 'swin_large_patch4_window7_224.ms_in22k_ft_in1k' in name:
                    self.img_size = 224
                    print(f"✅ Config hardcoded for Swin Large")
                elif 'vit_large_patch14_dinov2.lvd142m' in name:
                    self.img_size = 518
                    print(f"✅ Config hardcoded for ViT Large DINOv2")
                elif 'convnext_base.fb_in22k_ft_in1k' in name:
                    self.img_size = 224
                    print(f"✅ Config hardcoded for ConvNeXt Base")
                elif 'vit_giant_patch14_dinov2.lvd142m' in name:
                    self.img_size = 518
                    print(f"✅ Config hardcoded for ViT Giant DINOv2")
                
        except Exception as e:
            raise RuntimeError(f"Failed to load model from {self.model_path}: {e}")
    
    def get_transform(self):
        """Get validation transform for this model."""
        return transforms.Compose([
            transforms.Resize((self.img_size, self.img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=self.mean, std=self.std)
        ])
    
    def to_device(self, device):
        """Move model to device."""
        self.model.to(device)
        self.model.eval()
        return self

In [ ]:
class BiomassModelEVA02(nn.Module):
    """EVA02 model architecture for ensemble."""
    
    def regression_head(self, name, in_features: int, dropout: float):
        self.name = name
        return nn.Sequential(
            nn.Linear(in_features, in_features // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(in_features // 2, 1)
        )
    
    def __init__(self, model_name, dropout=0.0):
        super().__init__()
        
        # Shared backbone
        self.backbone = timm.create_model(
            model_name,
            pretrained=False,  
            num_classes=0, 
            global_pool='avg'  
        )
        n_features = int(self.backbone.num_features)
        n_features *= 2
        
        self.head_total = self.regression_head('dry_total', n_features, dropout)
        self.head_gdm = self.regression_head('dry_gdm', n_features, dropout)
        self.head_green = self.regression_head('dry_green', n_features, dropout)
    
    def forward(self, img_left, img_right):
        # Extract image features
        fl = self.backbone(img_left)
        fr = self.backbone(img_right)
        img_feat = torch.cat([fl, fr], dim=1)
        
        dry_total = self.head_total(img_feat)
        gdm = self.head_gdm(img_feat)
        dry_green = self.head_green(img_feat)
        
        # Return in expected format [B, 3]: [Green, Total, GDM]
        return torch.cat([dry_green, dry_total, gdm], dim=1)

In [ ]:
def load_eva02_model(model_path: str, device):
    """Load EVA02 model from checkpoint."""
    ckpt = torch.load(model_path, map_location=device)
    
    model_name = ckpt['model_name']
    val_score = ckpt['val_r2_score']
    state_dict = ckpt['model_state_dict']
    
    # Create model and load weights
    model = BiomassModelEVA02(model_name=model_name)
    model.load_state_dict(state_dict)
    model.eval()
    model.to(device)
    
    print(f"✅ Loaded EVA02 model with val R²: {val_score:.5f}")
    
    return model

In [19]:
class EnsembleDataset(Dataset):
    """Dataset that can apply different transforms per model."""
    
    def __init__(self, df: pd.DataFrame, img_dir: str, model_configs: list[ModelConfig]):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.model_configs = model_configs
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Load image once
        img_path = os.path.join(
            self.img_dir, row['image_path'].replace('test/', ''))
        image = cv2.imread(img_path)
        
        if image is None:
            raise FileNotFoundError(f"Cannot load image: {img_path}")
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Split into patches
        h, w, c = image.shape
        mid_w = w // 2
        left_patch = image[:, :mid_w, :]
        right_patch = image[:, mid_w:, :]
        
        # Apply transforms for each model
        batch = {
            'image_id': row['image_path'].split('/')[-1].replace('.jpg', ''),
            'models': []
        }
        
        for config in self.model_configs:
            left_pil = Image.fromarray(left_patch)
            right_pil = Image.fromarray(right_patch)
            
            transform = config.get_transform()
            left_tensor = transform(left_pil)
            right_tensor = transform(right_pil)
            
            batch['models'].append({
                'left_image': left_tensor,
                'right_image': right_tensor
            })
        
        return batch

In [ ]:
def predict_ensemble_batch(
    model_configs: list[ModelConfig], 
    batch: dict, 
    tta_types: list[str],
    ensemble_method: str = 'mean'  # 'mean' or 'weighted'
) -> torch.Tensor:
    """
    Run ensemble prediction with TTA for each model.
    
    Args:
        model_configs: List of model configurations
        batch: Batch from EnsembleDataset
        tta_types: List of TTA augmentations to apply
        ensemble_method: How to combine predictions ('mean' or 'weighted')
    
    Returns:
        Combined predictions [B, 5]
    """
    all_model_preds = []
    
    # Predict with each model
    for model_idx, config in enumerate(model_configs):
        model = config.model
        model_batch = batch['models'][model_idx]
        
        # Get tensors for this model
        left = model_batch['left_image'].to(DEVICE)
        right = model_batch['right_image'].to(DEVICE)
        
        # TTA predictions for this model
        tta_preds = []
        for tta in tta_types:
            left_t, right_t = apply_tta(left, right, tta)
            
            with torch.no_grad():
                preds_3 = model(left_t, right_t)  # [B, 3]
                preds_5 = expand_predictions(preds_3)  # [B, 5]
                tta_preds.append(preds_5)
        
        # Average across TTA for this model
        model_pred = torch.stack(tta_preds, dim=0).mean(dim=0)  # [B, 5]
        all_model_preds.append(model_pred)
    
    # Combine predictions from all models
    all_model_preds = torch.stack(all_model_preds, dim=0)  # [N_models, B, 5]
    
    if ensemble_method == 'mean':
        # Simple average
        final_pred = all_model_preds.mean(dim=0)  # [B, 5]
    elif ensemble_method == 'weighted':
        # Weighted average (can be customized based on validation scores)
        weights = torch.tensor([1.0] * len(model_configs), device=DEVICE)
        weights = weights / weights.sum()
        weights = weights.view(-1, 1, 1)  # [N_models, 1, 1]
        final_pred = (all_model_preds * weights).sum(dim=0)  # [B, 5]
    else:
        raise ValueError(f"Unknown ensemble method: {ensemble_method}")
    
    return final_pred

In [21]:
def run_ensemble_inference(
    model_configs: list[ModelConfig],
    test_df: pd.DataFrame,
    img_dir: str,
    batch_size: int = 8,
    tta_types: list[str] = ['id', 'hflip', 'vflip', 'hvflip'],
    ensemble_method: str = 'mean',
    num_workers: int = 0
) -> tuple[np.ndarray, list[str]]:
    """
    Run full ensemble inference pipeline.
    
    Args:
        model_configs: List of ModelConfig objects
        test_df: Test dataframe
        img_dir: Path to test images
        batch_size: Batch size for inference
        tta_types: TTA augmentations to apply
        ensemble_method: How to combine model predictions
        num_workers: Number of dataloader workers
    
    Returns:
        Tuple of (predictions array [N, 5], image_ids list)
    """
    # Move all models to device
    print("Loading models to device...")
    for i, config in enumerate(model_configs):
        config.to_device(DEVICE)
        print(f"✅ Model {i+1}/{len(model_configs)}: {config.model_path.split('/')[-1]}")
        print(f"   Size: {config.img_size}, Mean: {config.mean}, Std: {config.std}")
    
    # Create dataset and dataloader
    print("\nCreating ensemble dataloader...")
    test_dataset = EnsembleDataset(
        df=test_df,
        img_dir=img_dir,
        model_configs=model_configs
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    print(f"Dataloader ready: {len(test_loader)} batches")
    
    # Run inference
    all_predictions = []
    all_image_ids = []
    
    print(f"\nRunning ensemble inference with {len(tta_types)} TTA variants...")
    for batch in tqdm(test_loader, desc="Ensemble Inference"):
        # Predict with ensemble + TTA
        preds = predict_ensemble_batch(
            model_configs, batch, tta_types, ensemble_method
        )
        
        all_predictions.append(preds.cpu().numpy())
        all_image_ids.extend(batch['image_id'])
    
    # Concatenate results
    predictions_array = np.concatenate(all_predictions, axis=0)
    
    print(f"\n✅ Inference complete!")
    print(f"   Predictions shape: {predictions_array.shape}")
    print(f"   Images processed: {len(all_image_ids)}")
    
    return predictions_array, all_image_ids

### Predict

In [22]:
all_predictions = []
all_image_ids = []

In [23]:
os.listdir(MODELS_PATH)

['vit_large_patch14_dinov2.lvd142m-fold4-r2_0.7603.pt',
 'swin_large_patch4_window7_224.ms_in22k_ft_in1k-fold1-r2_0.7471.pt',
 'convnext_base.fb_in22k_ft_in1k-fold0-r2_0.7422.pt']

In [ ]:
if IS_ENSEMBLE:
    ensemble_configs = [
        ModelConfig(model_path=os.path.join(MODELS_PATH, fname))
        for fname in os.listdir(MODELS_PATH)
    ]

    # Run ensemble inference
    all_predictions_array, all_image_ids = run_ensemble_inference(
        model_configs=ensemble_configs,
        test_df=test_pivot,
        img_dir=PATH_TEST_IMG,
        batch_size=BATCH_SIZE,
        tta_types=TTA_TYPES,
        ensemble_method='mean',  # or 'weighted'
        num_workers=NUM_WORKERS
    )
else:
    model_config = ModelConfig(
        model_path=os.path.join(MODELS_PATH, best_model)
    )

    test_dataset = BiomassTestDataset(
        df=test_pivot,
        img_dir=PATH_TEST_IMG,
        transform=model_config.get_transform()
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS
    )
    # Single model inference 
    model = torch.load(
        os.path.join(MODELS_PATH, best_model),
        map_location=DEVICE,
        weights_only=False
    )
    model.to(DEVICE)
    model.eval()
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Inference"):
            # TTA predictions [B, 5]
            model_preds = predict_model_batch(model, batch, TTA_TYPES)
            
            all_predictions.append(model_preds.cpu().numpy())
            all_image_ids.extend(batch['image_id'])

    # Concatenate
    all_predictions_array = np.concatenate(all_predictions, axis=0)

# If using EVA02, add its predictions to ensemble
if IS_USE_EVA:
    print("Adding EVA02 model to predictions...")
    
    # Load EVA02 model
    eva_model = load_eva02_model(EVA02_MODEL_PATH, DEVICE)
    
    # Create EVA02 dataset with different image size
    eva_transform = transforms.Compose([
        transforms.Resize((EVA02_IMG_SIZE, EVA02_IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    eva_dataset = BiomassTestDataset(
        df=test_pivot,
        img_dir=PATH_TEST_IMG,
        transform=eva_transform
    )
    
    eva_loader = DataLoader(
        eva_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS
    )
    
    # Get EVA02 predictions
    eva_predictions = []
    with torch.no_grad():
        for batch in tqdm(eva_loader, desc="EVA02 Inference"):
            eva_preds = predict_model_batch(eva_model, batch, TTA_TYPES)
            eva_predictions.append(eva_preds.cpu().numpy())
    
    eva_predictions_array = np.concatenate(eva_predictions, axis=0)
    
    # Apply EVA02-specific post-processing
    print("\nApplying EVA02 post-processing...")
    
    # Extract individual predictions [clover, dead, green, total, gdm]
    clover = eva_predictions_array[:, 0]
    dead = eva_predictions_array[:, 1]
    green = eva_predictions_array[:, 2]
    total = eva_predictions_array[:, 3]
    gdm = eva_predictions_array[:, 4]
    
    # Apply smoothing constraints
    clover = np.clip(clover, 0, EVA02_SMOOTH_FACTOR * gdm)
    dead = np.clip(dead, 0, EVA02_SMOOTH_FACTOR * total)
    
    # Apply minimum thresholds
    clover = np.where(clover < EVA02_DRY_CLOVER_MIN, 0, clover)
    dead = np.where(dead < EVA02_DRY_DEAD_MIN, 0, dead)
    
    # Reconstruct predictions
    eva_predictions_array[:, 0] = clover
    eva_predictions_array[:, 1] = dead
    
    # Ensemble: average both predictions
    print(f"\nEnsembling predictions...")
    print(f"  Main model shape: {all_predictions_array.shape}")
    print(f"  EVA02 shape: {eva_predictions_array.shape}")
    
    # Simple average ensemble
    all_predictions_array = (all_predictions_array + eva_predictions_array) / 2.0
    
    print(f"✅ Final ensemble shape: {all_predictions_array.shape}")



print(f"Predictions shape: {all_predictions_array.shape}")
print(f"Image IDs count: {len(all_image_ids)}")

/usr/local/lib/python3.12/dist-packages/torch/serialization.py:1493: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn(


✅ Config hardcoded for ViT Large DINOv2


Inference: 100%|██████████| 1/1 [00:06<00:00,  6.80s/it]

Predictions shape: (1, 5)
Image IDs count: 1


In [25]:
# Format submission CSV
# Columns order: Dry_Clover_g, Dry_Dead_g, Dry_Green_g, Dry_Total_g, GDM_g
target_names = ['Dry_Clover_g', 'Dry_Dead_g',
                'Dry_Green_g', 'Dry_Total_g', 'GDM_g']

submission_rows = []

for img_idx, image_id in enumerate(all_image_ids):
    predictions = all_predictions_array[img_idx]  # [5] values for 5 targets

    for target_idx, target_name in enumerate(target_names):
        sample_id = f"{image_id}__{target_name}"
        target_value = float(predictions[target_idx])

        submission_rows.append({
            'sample_id': sample_id,
            'target': target_value
        })

# Create submission dataframe
submission_df = pd.DataFrame(submission_rows)

print(f"Submission shape: {submission_df.shape}")
print(f"Expected shape: ({len(test_pivot) * 5}, 2)")
print(submission_df.head(10))

Submission shape: (5, 2)
Expected shape: (5, 2)
                    sample_id     target
0  ID1001187975__Dry_Clover_g   0.000000
1    ID1001187975__Dry_Dead_g  33.334351
2   ID1001187975__Dry_Green_g  49.095665
3   ID1001187975__Dry_Total_g  77.252182
4         ID1001187975__GDM_g  43.917839


In [26]:
SUBMISSION_NAME = 'submission.csv'

In [27]:
# Save submission
submission_df.to_csv(SUBMISSION_NAME, index=False)

print(f"Submission saved to: {SUBMISSION_NAME}")

Submission saved to: submission.csv
